In [1]:
# Dataset: 10.5281/zenodo.1048301
from marsvision.utilities import DataLoader

In [26]:
import os
# Dataset location
# Adjust to point to root of dataset
dataset_root = "X:\hirise-map-proj"

In [34]:
# Apply the SKLearn pipeline--
# Load data, extract features, train a model.
data_loader = DataLoader(os.path.join(dataset_root, "map-proj"))
data_loader.data_reader()
data_loader.data_transformer() 
# The extracted features we're training on should be in data_loader.df

# Shaping dataframe to contain features and class codes
lines = pd.read_csv(os.path.join(dataset_root, "labels-map-proj.txt"), 
                    delimiter=" ",  
                    header = None,
                   names=["file_name", "class_code"])
lines = lines.sort_values(by="file_name")

df_labelled = lines.join(data_df.set_index("file_name"), 
                         on="file_name",
                         lsuffix="_left",
                         rsuffix="_right")
df_labelled = df_labelled.drop(labels=["class_code_right", "class"], axis=1)
df_labelled = df_labelled.rename(columns={
    "file_name" : "file_name", 
    "class_code_left" : "class_code"})



In [44]:
df_labelled

,file_name,class_code,0,1,2,3,4,5
873,ESP_011283_2265_RED-0032.jpg,0,84.706379,14424.955996,0.002600,403.657701,102.938210,1758.408455
465,ESP_011289_1950_RED-0006.jpg,6,3.266122,822.193554,-0.001553,45.913871,78.866910,5354.609701
1815,ESP_011289_1950_RED-0007.jpg,6,5.141668,1284.688541,-0.001397,57.873429,60.246386,4771.687281
584,ESP_011289_1950_RED-0009.jpg,6,2.192261,554.220464,-0.000272,49.464534,75.727784,5847.890326
1397,ESP_011289_1950_RED-0013.jpg,6,2.810844,708.864470,0.001979,112.121442,77.699237,5529.686793
...,...,...,...,...,...,...,...,...
1377,TRA_000828_1805_RED-0148.jpg,6,15.058802,3613.226956,0.000252,52.019892,71.678802,5660.806630
513,TRA_000828_1805_RED-0149.jpg,6,11.644220,2833.688185,-0.002193,43.480424,85.822217,5167.247265
2478,TRA_000828_1805_RED-0152.jpg,0,12.139087,2948.109689,-0.001669,16.581029,112.054319,1032.092588
1577,TRA_000828_1805_RED-0154.jpg,0,12.272701,2978.919521,-0.007510,35.245009,82.779309,4322.291647


In [53]:
x = df_labelled.iloc[:, 2:8]
y = df_labelled["class_code"]

In [61]:
from sklearn.model_selection import StratifiedKFold, cross_validate
def cv_scores(model, x, y, label, cv, groups=None):
    model_cv_results = cross_validate(model, x, y, cv=cv, groups=groups, 
                        scoring="accuracy")
    model_cv_acc_mean = np.mean(model_cv_results["test_score"])
    print(label)
    print("Accuracy: " + str(model_cv_acc_mean))

In [57]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 500)

In [63]:
sfold = StratifiedKFold(5, shuffle=True)
cv_scores(random_forest, x, y, "Random Forest Classifier", sfold)

Random Forest Classifier
Accuracy: 0.7607329842931938
